In [2]:
from datetime import datetime
import networkx as nx
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

In [3]:
class graph:
    
    # model parameters
    m = None
    delta_repr = None # this is delta / m

    # graph state
    current_timestep = 0        

    '''
    initializes the PAM
    @param m >=1, number of edges added each timestep
    @param delta >= -m, determines the growth rule
    @param t >= 1, number of timesteps
    @argument recursive=False, there are no benefits to recursive implementation, it is limited by the recursion depth. 
                                but I'm still keeping it in, sue me.
    @constraint: delta >= -m
    '''
    def __init__(self, m, delta, t):
        assert delta >= -m, "parameter delta has to be >= -m"

        self.delta_repr = delta / m
        self.m = m
        
        #political orientation
        self.ors = np.array(['gop', 'ind', 'dem'])
        self.votes = np.zeros(t).astype('str')

        # the representation of the PAM in terms of PAM with m = 1.
        self.G_repr = nx.Graph()

        #self.degr_distr = {}
        self.degr_distr = np.array([])
        
        self.iterativeGrowth(t)

    def growFurther(self, t):
        assert t >= self.current_timestep, "timestep given in recursiveGrowth is less than current timestep."
        self.iterativeGrowth(t)
        
    def growCoex(self, t):
        assert t >= self.current_timestep
        self.coexGrowth(t) 
        
    '''
    iteratively grow G_repr until it has self.m*t nodes
    '''
    def iterativeGrowth(self, t):
        for i in tqdm(range(self.G_repr.number_of_nodes(), self.m*t)):
            # add new vertex (first vertex is called "0")
            self.G_repr.add_node(i)

            # add the appropriate edge with the heuristic  (8.2.1)
            self._addConnection()

        self.current_timestep = t
        self._collapseGraph()
        
    def coexGrowth(self, t):
        for i in tqdm(range(self.G.number_of_nodes(), t)):
            # add new vertex (first vertex is called "0")
            self.G_repr.add_nodes_from(np.arange(self.m) + self.m*i)
            
            # add the appropriate edge with the heuristic  (8.2.1)
            self._coexConnection()

        self.current_timestep = t
        self._coexcollapseGraph()
        

    def _addConnection(self):
        # self loop probability
        dists = np.append(self.degr_distr, 1 + self.delta_repr)
        # select randomly the name of another vertex
        secondEndpoint = np.random.choice(len(dists), p = dists/np.sum(dists))
                      
        # maintain degree distr
        self.degr_distr = dists
        self.degr_distr[secondEndpoint] += 1

        # update self.G
        self.G_repr.add_edge(len(dists)-1, secondEndpoint)
        
    def _coexConnection(self):
        choices = np.array([])
        for i in range(self.m):
            # self loop probability
            dists = np.append(self.degr_distr, 1 + self.delta_repr)

            # select randomly the name of another vertex
            secondEndpoint = np.random.choice(len(dists), p = dists/np.sum(dists))

            #copy the type of the other vertex
            if secondEndpoint//self.m < len(self.votes):
                choices = np.append(choices, self.votes[secondEndpoint//self.m])
            else:
                choices = np.append(choices, 'ind') 
            
            # maintain degree distr
            self.degr_distr = dists
            self.degr_distr[secondEndpoint] += 1

            # update self.G
            self.G_repr.add_edge(len(dists)-1, secondEndpoint)
        vote = np.append(self.votes, np.random.choice(choices, p = np.ones(len(choices))/len(choices)))
        self.votes = vote
        
    

    ''' 
    Explictely calculate the probabilities of attachment for each node. 
    '''
    def _calculateConnectionPMF(self):
        local_t = self.G_repr.number_of_nodes() - 1
        probabilities = np.zeros(self.G_repr.number_of_nodes())

        # self loop probability  (1 + δ)/(local_t(2 + δ) + (1 + δ))
        probabilities[-1] = (1 + self.delta_repr) / (local_t*(2 + self.delta_repr) + (1 + self.delta_repr))

        # other vertices probabilities:  (vertex_degr + δ)/(local_t(2 + δ) + (1 + δ))
        for i in range(len(probabilities)-1):

            # find degree of vertex with a self loop counting double
            adj = self.G_repr.adj[i] 
            vertex_degr = len(adj)+1 if (i in adj.keys()) else len(adj)

            probabilities[i]  = (vertex_degr + self.delta_repr)/(local_t*(2 + self.delta_repr) + (1 + self.delta_repr))

        
        assert abs(sum(probabilities) - 1) < 0.00001, "PMF does not sum to 1, but to {}".format(sum(probabilities))

        return probabilities

    '''
    (PAM_m,delta)_t is defined in terms of(PAM_1,delta/m)_mt.
    Thus to get the (PAM_m,delta)_t, the alternative representation has to be collapsed.
    '''
    def _collapseGraph1(self):
        if self.m == 1:
            self.G = self.G_repr
        else:
            # collapses G_repr into G
            self.G = nx.Graph()

            # add t vertices 
            for i in range(self.current_timestep):
                self.G.add_node(i)
            
            for i in range(self.current_timestep): # i denotes the collapsed vertex
                for j in range(self.m): # j iterates through 
                    vertex_index = i*self.m + j

                    # loop through the adjacent vertices
                    for adj_vertex in self.G_repr.adj[vertex_index].keys():

                        # find the name of the collapsed vertex
                        adj_vertex_collapsed = (adj_vertex // self.m)

                        # add edge to the collapsed graph
                        self.G.add_edge(i, adj_vertex_collapsed)

    def _collapseGraph(self):
        self.G = nx.Graph()
        self.G.add_nodes_from(np.arange(self.current_timestep))
        for i in range(self.current_timestep):
            self.votes[i] = np.random.choice(self.ors, p = [1/3, 1/3, 1/3])
            self.G.nodes[i]["type"] = self.votes[i]
            nodes = np.arange(0, self.m) + self.m*i 
            edges = list(self.G_repr.edges(nodes))
            for edge in edges:
                self.G.add_edge(i, edge[1] // self.m)
                
    def _coexcollapseGraph(self):
        self.G = nx.Graph()
        self.G.add_nodes_from(np.arange(self.current_timestep))
        for i in range(self.current_timestep):
            self.G.nodes[i]["type"] = self.votes[i]
            nodes = np.arange(0, self.m) + self.m*i 
            edges = list(self.G_repr.edges(nodes))
            for edge in edges:
                self.G.add_edge(i, edge[1] // self.m)
                
    
    def draw(self):
        nx.draw_circular(self.G)
        plt.show()
        
    def draw2(self):
        nx.draw_circular(self.G_repr)
        plt.show()
        
    def draw3(self):
        color_state_map = {'gop': 'red', 'ind': 'yellow', 'dem': 'blue'}
        label = {}
        for i in range(self.current_timestep):
            label[i] = self.G.nodes[i]["type"]
        nx.draw_networkx(self.G, node_color=[color_state_map[node[1]['type']] 
                    for node in self.G.nodes(data=True)], with_labels = True, labels = label)
        plt.show()

    def draw4(self):
        color_state_map = {'gop': 'red', 'ind': 'yellow', 'dem': 'blue'}
        nx.draw_networkx(self.G, node_color=[color_state_map[node[1]['type']] 
                    for node in self.G.nodes(data=True)], with_labels=False)
        plt.show()
    '''
    Returns distribution of typical distance:
    - the length of the shortest path between two randomly drawn nodes, given that they are connected
    @param sample: The number of randomly drawn 
    '''
    def typicalDistanceDistribution(self, sample=-1):

        all_shortest_paths = []
        if sample == -1:
            #dictionary of dictionaries dict[source][target] = path
            for source, destinations in nx.algorithms.shortest_path(self.G).items():
                for destination, path in destinations.items():
                    all_shortest_paths.append(path)
        else:
            sources = random.choices(list(self.G.nodes), k=sample)
            targets = random.choices(list(self.G.nodes), k=sample)
            for i in range(sample):
                all_shortest_paths.append(nx.algorithms.shortest_path(self.G, sources[i], targets[i]))

        # calculate pmf
        pmf = {}
        numberOfPaths = 0 #if sample > 0, then this will end up being equal to sample
        for path in all_shortest_paths:
            if (len(path)-1) in pmf:
                pmf[len(path)-1] += 1
            else: 
                pmf[len(path)-1] = 1
            numberOfPaths += 1

        print(numberOfPaths)

        #normalize the histogram (paths currently double counted)
        for key in pmf.keys():
            pmf[key] = pmf[key] / numberOfPaths

        assert abs(sum([v for v in pmf.values()]) - 1) < 0.00001, "pmf does not sum to one!!"

        return pmf

    '''
    Returns size of largest connected component (giant components)
    Note: Strictly speaking, we assume the GRG is highly connected, that is, as n -> \inf, 
    liminf of the ( size of the largest connected component / size of network) > 0.
    '''
    def getSizeOfGiantComponent(self):
        # get sorted list of size of all connected components
        component_sizes = [len(c) for c in sorted(nx.connected_components(self.G), key=len, reverse=True)]
        return component_sizes[0]

    '''
    store the networkx graph object
    '''
    def dumpGraph(self, fp='default'):
        import pickle

        if fp == 'default':
            fp = datetime.now().strftime("%H:%M:%S") + ".graph"
        elif not ".graph" in fp:
            fp = fp + ".graph"

        with open(fp, 'wb') as f:
            pickle.dump(self, f)

            
#ideetje al collapsen na de edge toe te voegen

In [4]:
pam = graph(2, 0, 10)

100%|██████████| 20/20 [00:00<00:00, 6721.64it/s]


In [5]:
pam.growCoex(1000)

100%|██████████| 990/990 [00:00<00:00, 3986.56it/s]


In [6]:
pam.getSizeOfGiantComponent()

#hello!

1000

In [ ]:
def _collapseGraph2(self):
    adj = nx.to_numpy_array(self.G_repr).astype('int8')
    adjslice = np.lib.stride_tricks.as_strided(adj, shape=(self.current_timestep,self.current_timestep,self.m,self.m), strides=(2*self.current_timestep*self.m,self.m,self.current_timestep*self.m,1))
    newadj = np.zeros((self.current_timestep,self.current_timestep))
    for i in range(self.current_timestep):
        for j in range(self.current_timestep):
            newadj[i][j] = np.max(adjslice[i][j])
    self.G = nx.from_numpy_array(newadj)
        
def merge_nodes(G,nodes, new_node, attr_dict=None, **attr):
    
    G.add_node(new_node, attr_dict, **attr) # Add the 'merged' node
    
    for n1,n2,data in G.edges(data=True):
        # For all edges related to one of the nodes to merge,
        # make an edge going to or coming from the `new gene`.
        if n1 in nodes:
            G.add_edge(new_node,n2,data)
        elif n2 in nodes:
            G.add_edge(n1,new_node,data)
    
    for n in nodes: # remove the merged nodes
        G.remove_node(n)